### Import necessary functions

In [1]:
import os
import sys
import re
import random
import tempfile
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as fn
import networkx as nx
import plotly.graph_objects as go
import plotly.io as pio
from arango import ArangoClient
import nx_arangodb as nxadb
from langchain_community.graphs import ArangoGraph
from langchain.chat_models import ChatOpenAI
from langchain.agents import ZeroShotAgent, AgentExecutor, Tool, AgentType, initialize_agent, AgentOutputParser
from langchain.chains import LLMChain, GraphQAChain, ArangoGraphQAChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction, AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from langchain.tools import Tool
from langchain_core.tools import tool
from IPython.display import display, HTML
from openai import OpenAI
from typing import List, Dict

[21:27:38 +0000] [INFO]: NetworkX-cuGraph is available.


### Change networkx backend to cugraph for hardware acceleration

In [2]:
!NETWORKX_BACKEND_PRIORITY=cugraph

### Configuration

In [ ]:
NODES_PATH = "nodes.csv"
EDGES_PATH = "edges.csv"
ENTITY_EMB_PATH = r"embed\DRKG_TransE_l2_entity.npy"
REL_EMB_PATH = r"embed\DRKG_TransE_l2_relation.npy"
ENTITY_IDMAP_PATH = r"embed\entities.tsv"
RELATION_IDMAP_PATH = r"embed\relations.tsv"
DRUG_LIST_PATH = "infer_drug.tsv"
DRUG_VOCAB_PATH = "drugbank vocabulary.csv"

HOSTS = "https://30391f9d007a.arangodb.cloud:8529"
USERNAME = "view-only"

In [ ]:
PASSWORD = "viewdb"
op_pwd = os.environ["OPENAI_API_KEY"]

### Define functions

In [5]:
def load_graph_data(nodes_path: str, edges_path: str) -> nx.Graph:
    """
    Load nodes and edges from CSV files and create a NetworkX graph.
    
    Args:
        nodes_path (str): Path to nodes CSV file
        edges_path (str): Path to edges CSV file
    
    Returns:
        nx.Graph: A graph with nodes and edges
    """
    # Read CSV files
    nodes_df = pd.read_csv(nodes_path)
    edges_df = pd.read_csv(edges_path)
    
    # Create an empty graph
    G = nx.Graph()
    
    # Add nodes with attributes
    for node, attrs in nodes_df.iterrows():
        node_attrs = attrs.dropna().to_dict()
        G.add_node(node, **node_attrs)
    
    # Add edges with attributes
    for _, row in edges_df.iterrows():
        attrs = eval(row['attributes']) if isinstance(row['attributes'], str) else row['attributes']
        G.add_edge(row['source'], row['target'], **attrs)
    
    return G

def create_arangodb_connection(hosts: str, username: str, password: str, verify: bool = True):
    """
    Create an ArangoDB database connection.
    
    Args:
        hosts (str): ArangoDB host URL
        username (str): Database username
        password (str): Database password
        verify (bool, optional): Whether to verify connection. Defaults to True.
    
    Returns:
        ArangoDB database connection
    """
    client = ArangoClient(hosts=hosts)
    return client.db(username=username, password=password, verify=verify)

def create_arangosearch_view(db, view_name: str, collection_name: str, indexed_fields: Dict[str, List[str]]):
    """
    Create an ArangoSearch view for a specific collection.
    
    Args:
        db: ArangoDB database connection
        view_name (str): Name of the ArangoSearch view
        collection_name (str): Name of the collection to be linked
        indexed_fields (dict): Dictionary of fields and their analyzers
    """
    # Fetch existing views
    existing_views = [view["name"] for view in db.views()]
    
    if view_name not in existing_views:
        db.create_view(
            name=view_name,
            view_type="arangosearch",
            properties={
                "cleanupIntervalStep": 2,
                "consolidationIntervalMsec": 60000
            }
        )
        print(f"Created ArangoSearch view: {view_name}")
    else:
        print(f"View '{view_name}' already exists.")

    # Define link properties
    link_properties = {
        "links": {
            collection_name: {
                "includeAllFields": False,
                "fields": {field: {"analyzers": analyzers} for field, analyzers in indexed_fields.items()}
            }
        }
    }
    
    # Update the view with the collection link
    db.update_view(name=view_name, properties=link_properties)
    print(f"Linked collection '{collection_name}' to view '{view_name}' with indexed fields: {indexed_fields}")

def search_arangodb_view(db, view_name: str, search_fields: List[str], 
                         search_term: str, filters: Dict = None, 
                         limit: int = 10, offset: int = 0, 
                         sort_field: str = None, sort_direction: str = "ASC"):
    """
    Perform a generic search on an ArangoDB view.
    
    Args:
        db: ArangoDB database connection
        view_name (str): Name of the view to search
        search_fields (list): Fields to search within
        search_term (str): Term to search for
        filters (dict, optional): Field-value pairs to filter results
        limit (int, optional): Maximum number of results. Defaults to 10.
        offset (int, optional): Number of results to skip. Defaults to 0.
        sort_field (str, optional): Field to sort results by
        sort_direction (str, optional): Sort direction. Defaults to "ASC".
    
    Returns:
        list: Search results
    """
    # Start building the AQL query
    aql_query = f"""
    FOR doc IN `{view_name}`
        SEARCH ANALYZER(
    """
    
    # Build search conditions
    search_conditions = [f"PHRASE(doc.{field}, @search_term, \"text_en\")" for field in search_fields]
    aql_query += " OR ".join(search_conditions)
    aql_query += ", \"text_en\")"
    
    # Add filters
    if filters and isinstance(filters, dict) and len(filters) > 0:
        filter_conditions = [f"doc.{field} == @{field}" for field, value in filters.items()]
        filter_str = " AND ".join(filter_conditions)
        aql_query += f"\n        FILTER {filter_str}"
    
    # Add sorting
    if sort_field:
        aql_query += f"\n        SORT doc.{sort_field} {sort_direction}"
    
    # Add limit and offset
    aql_query += f"\n        LIMIT {offset}, {limit}"
    
    # Finish the query
    aql_query += "\n        RETURN doc"
    
    # Prepare bind variables
    bind_vars = {"search_term": search_term}
    if filters and isinstance(filters, dict):
        bind_vars.update(filters)
    
    # Execute the query
    try:
        cursor = db.aql.execute(aql_query, bind_vars=bind_vars)
        results = [doc for doc in cursor]
        return results
    except Exception as e:
        print(f"Error executing AQL query: {e}")
        print(f"Query: {aql_query}")
        print(f"Bind variables: {bind_vars}")
        return []

In [6]:
def graph_to_text(subgraph, drug_node, disease_node):
    """
    Convert subgraph data into natural language text descriptions using GPT.

    :param subgraph: NetworkX graph object containing the subgraph
    :param drug_node: ID of the drug node
    :param disease_node: ID of the disease node
    :return: Natural language text description
    """
    # Initialize OpenAI client
    client = OpenAI()
    
    # Extract edges and their attributes
    edges = subgraph.edges(data=True)
    descriptions = []

    for edge in edges:
        source, target, attrs = edge
        description = f"{source} is connected to {target} with attributes {attrs}."
        descriptions.append(description)

    # Combine all descriptions into a single text
    combined_text = " ".join(descriptions)
    
    # Create prompt for GPT
    prompt = f"Convert this graph information into a natural language description:\n{combined_text}"
    
    # Use OpenAI API to generate description
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system", 
                "content": "You are a helpful assistant that converts graph relationships into natural language descriptions."
            }, 
            {
                "role": "user", 
                "content": prompt
            }
        ],
        temperature=1.0,
        max_tokens=150
    )
    
    return response.choices[0].message.content.strip()
def generate_explanation(prompt: str) -> str:
    """
    Generate explanation using OpenAI's language model.

    :param prompt: Prompt text to generate explanation
    :return: Generated explanation text
    """
    client = OpenAI()
    
    response = client.chat.completions.create(
        model="gpt-4o",  # or another appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful assistant that explains medical relationships."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        n=1,
        temperature=1.0,
    )
    return response.choices[0].message.content.strip()

def create_prompt(subgraph, drug_node, disease_node) -> str:
    """
    Create prompt for generating explanations from graph context.

    :param subgraph: NetworkX graph object containing the subgraph
    :param drug_node: ID of the drug node
    :param disease_node: ID of the disease node
    :return: Prompt text
    """
    edges = subgraph.edges(data=True)
    descriptions = []

    for edge in edges:
        source, target, attrs = edge
        description = f"{source} is connected to {target} with attributes {attrs}."
        descriptions.append(description)

    combined_text = " ".join(descriptions)
    prompt = (
        f"Explain why {drug_node} is a good treatment for {disease_node} based on the following relationships:\n"
        f"{combined_text}\n"
        f"Remember the name of disease node {disease_node} is  {disease_name} and the name of the drug node {drug_node} is {drug_name}"
        "Provide a detailed explanation."
    )
    return prompt

def extract_disease_name(query, api_key=op_pwd):
    """
    Extract the specific disease name from a medical query.
    
    Parameters:
    -----------
    query : str
        The original query containing a disease name
    api_key : str, optional
        OpenAI API key. Defaults to op_pwd from the global scope.
    
    Returns:
    --------
    str
        Extracted disease name
    """
    client = OpenAI(api_key=api_key)
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system", 
                    "content": "You are a precise medical information extractor. Extract ONLY the specific disease name from the given query. If multiple disease names are present, choose the most specific one."
                },
                {
                    "role": "user", 
                    "content": f"Extract the exact disease name from this query:\n\n{query}"
                }
            ],
            max_tokens=50,
            n=1,
            stop=None,
            temperature=1.0
        )
        
        disease_name = response.choices[0].message.content.strip()
        return disease_name
    
    except Exception as e:
        print(f"Error in extracting disease name: {e}")
        return None

In [7]:
def load_embedding_data(entity_path: str, relation_path: str, 
                        entity_idmap_path: str, relation_idmap_path: str):
    """
    Load embedding data and create mappings.
    
    Args:
        entity_path (str): Path to entity embeddings
        relation_path (str): Path to relation embeddings
        entity_idmap_path (str): Path to entity ID mapping file
        relation_idmap_path (str): Path to relation ID mapping file
    
    Returns:
        Tuple of (entity embeddings, relation embeddings, entity maps, relation map)
    """
    # Load embeddings
    entity_emb = np.load(entity_path)
    rel_emb = np.load(relation_path)
    
    # Create mappings
    entity_map, entity_id_map = {}, {}
    relation_map = {}
    
    with open(entity_idmap_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
        for row_val in reader:
            entity_map[row_val['name']] = int(row_val['id'])
            entity_id_map[int(row_val['id'])] = row_val['name']
    
    with open(relation_idmap_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
        for row_val in reader:
            relation_map[row_val['name']] = int(row_val['id'])
    
    return entity_emb, rel_emb, entity_map, entity_id_map, relation_map
def transE_l2_scoring(drug_emb, treatment_emb, disease_emb, gamma=12.0):
    """
    Compute TransE L2 scoring for drug-disease relationships.
    
    Args:
        drug_emb (torch.Tensor): Drug embeddings
        treatment_emb (torch.Tensor): Treatment relation embeddings
        disease_emb (torch.Tensor): Disease embeddings
        gamma (float, optional): Margin parameter. Defaults to 12.0.
    
    Returns:
        torch.Tensor: Scoring of drug-disease relationships
    """
    score = drug_emb + treatment_emb - disease_emb
    return gamma - torch.norm(score, p=2, dim=-1)
def predict_treatments(drug_ids, disease_ids, treatment_rid, 
                       entity_emb, rel_emb, 
                       entity_id_map, clinical_drug_map, 
                       topk=100):
    """
    Predict potential treatments for given diseases.
    
    Args:
        drug_ids (list): List of drug entity IDs
        disease_ids (list): List of disease entity IDs
        treatment_rid (list): List of treatment relation IDs
        entity_emb (np.ndarray): Entity embeddings
        rel_emb (np.ndarray): Relation embeddings
        entity_id_map (dict): Mapping of entity IDs to names
        clinical_drug_map (dict): Mapping of DrugBank IDs to common names
        topk (int, optional): Number of top predictions to return. Defaults to 100.
    
    Returns:
        pd.DataFrame: DataFrame of predicted treatments
    """
    drug_ids = torch.tensor(drug_ids).long()
    disease_ids = torch.tensor(disease_ids).long()
    treatment_rid = torch.tensor(treatment_rid)
    
    drug_emb = torch.tensor(entity_emb[drug_ids])
    treatment_embs = [torch.tensor(rel_emb[rid]) for rid in treatment_rid]
    
    scores_per_disease = []
    dids = []
    
    for rid in range(len(treatment_embs)):
        treatment_emb = treatment_embs[rid]
        for disease_id in disease_ids:
            disease_emb = torch.tensor(entity_emb[disease_id])
            score = fn.logsigmoid(transE_l2_scoring(drug_emb, treatment_emb, disease_emb))
            scores_per_disease.append(score)
            dids.append(drug_ids)
    
    scores = torch.cat(scores_per_disease)
    dids = torch.cat(dids)
    
    # Sort and filter unique predictions
    scores_tensor = torch.tensor(scores)
    idx = torch.flip(torch.argsort(scores_tensor), dims=[0])
    scores = scores_tensor[idx].numpy()
    dids = dids[idx.numpy()]
    
    # Get unique top-k predictions
    _, unique_indices = np.unique(dids, return_index=True)
    topk_indices = np.sort(unique_indices)[:topk]
    proposed_dids = dids[topk_indices]
    proposed_scores = scores[topk_indices]
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        'drug_id': [entity_id_map[int(did)] for did in proposed_dids],
        'score': proposed_scores,
        'original_index': range(len(proposed_dids))
    })
    
    # Add drug names from clinical trial drugs mapping
    results_df['drug_name'] = results_df['drug_id'].map(clinical_drug_map)
    
    return results_df


In [8]:
def get_candidates(query: str) -> str:
    """
    Identifies potential drug repurposing candidates for a given disease using knowledge graph embeddings.

    This tool takes a disease query as input, extracts the disease name, and searches an ArangoDB 
    graph database for related disease nodes. It then retrieves embeddings for entities and relationships 
    from precomputed knowledge graph data. Using these embeddings, the tool predicts possible drug candidates 
    that could be repurposed for treating the given disease. The predictions are filtered to include only 
    clinically relevant drugs.

    Args:
        query (str): A natural language query specifying a disease name.

    Returns:
        str: A formatted string containing a list of predicted drugs with clinical relevance.
    """
    global disease_list
    global clinical_results
    disease_name = extract_disease_name(query)
    print(disease_name) 
    disease_results = search_arangodb_view(
        db=db,
        view_name="node-search",
        search_fields=["name", "Identifier"],
        search_term=disease_name,
        filters={"type": "Disease"},
        limit=5
    )
    print(disease_results)
    disease_list = [result['Identifier'] for result in disease_results]

    entity_emb, rel_emb, entity_map, entity_id_map, relation_map = load_embedding_data(
        ENTITY_EMB_PATH, REL_EMB_PATH, 
        ENTITY_IDMAP_PATH, RELATION_IDMAP_PATH
        )

    disease_list = [result['Identifier'] for result in disease_results]

    entity_emb, rel_emb, entity_map, entity_id_map, relation_map = load_embedding_data(
        ENTITY_EMB_PATH, REL_EMB_PATH, 
        ENTITY_IDMAP_PATH, RELATION_IDMAP_PATH
    )

    with open(DRUG_LIST_PATH, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['drug','ids'])
        drug_list = [row_val['drug'] for row_val in reader]

    treatment = ['Hetionet::CtD::Compound:Disease', 'GNBR::T::Compound:Disease']

    drug_ids = [entity_map[drug] for drug in drug_list]
    disease_ids = [entity_map[disease] for disease in disease_list]
    treatment_rid = [relation_map[treat] for treat in treatment]

    drug_vocab = pd.read_csv(DRUG_VOCAB_PATH)
    clinical_drug_map = dict(zip(drug_vocab['DrugBank ID'], drug_vocab['Common name']))
    clinical_drug_map = {k: v for k, v in clinical_drug_map.items() if isinstance(v, str)}

    results_df = predict_treatments(
        drug_ids, disease_ids, treatment_rid, 
        entity_emb, rel_emb, 
        entity_id_map, clinical_drug_map
    )

    clinical_results = results_df[results_df['drug_name'].notnull()]
    print("\nClinical trial drugs in top predictions:")
    print(clinical_results)

In [9]:
@tool
def drug_repurposing(query: str) -> str:
    """
    Identifies potential drug repurposing candidates for a given disease and analyzes their relationship.

    This tool retrieves potential drug candidates for a specified disease using knowledge graph embeddings 
    and clinical trial data. It ranks the top candidates and analyzes the most relevant drug-disease relationship 
    to provide insights into the mechanism of action.

    Args:
        query (str): A natural language query specifying a disease name and asking for potential drug repurposing candidates.

    Returns:
        str: A formatted string containing a list of predicted drugs in clinical trials and their mechanism of action.
    """
    global disease_list
    global clinical_results
    
    # Retrieve drug candidates
    get_candidates(query)
    
    if clinical_results.empty:
        return "No clinical trial drugs found for the given disease."

    # Limit to top 5 drugs
    clinical_results = clinical_results[0:5]
    drug_list = clinical_results['drug_id']
    drug_id = drug_list[0]
    disease_id = disease_list[0]
    try:
    # Analyze drug-disease relationship
        result = analyze_drug_disease_relationship_aql(
            db=db,
            compound_id=drug_id,  
            disease_id=disease_id  
        )

    # Format output
        output = (
            f"\nDrug Information:\n"
            f"Name: {result['drug_info']['name']}\n"
            f"ID: {result['drug_info']['id']}\n\n"
            f"Disease Information:\n"
            f"Name: {result['disease_info']['name']}\n"
            f"ID: {result['disease_info']['id']}\n\n"
            f"Relationship Description:\n"
            f"{result['description']}\n\n"
            f"Mechanism of Action:\n"
            f"{result['explanation']}"
        )

        return output

    except ValueError as e:
        return f"Error: {e}"

In [10]:
def analyze_drug_disease_relationship(db, compound_id: str, disease_id: str) -> dict:
    """
    Analyzes the relationship between a compound and disease by extracting the subgraph,
    generating text descriptions and explanations of the mechanism of action.

    Args:
        db: ArangoDB database connection
        compound_id (str): Identifier for the compound/drug
        disease_id (str): Identifier for the disease
    
    Returns:
        dict: Dictionary containing relationship information
    """
    # Get drug node information
    drug_node = search_arangodb_view(
        db=db,
        view_name="node-search",
        search_fields=["Identifier"],
        search_term=compound_id,
        limit=1
    )
    if not drug_node:
        raise ValueError(f"No drug found with identifier: {compound_id}")
    
    drug_name = drug_node[0]['name']
    drug_node_id = drug_node[0]['_id']

    # Get disease node information 
    disease_node = search_arangodb_view(
        db=db,
        view_name="node-search", 
        search_fields=["Identifier"],
        search_term=disease_id,
        limit=1
    )
    if not disease_node:
        raise ValueError(f"No disease found with identifier: {disease_id}")

    disease_name = disease_node[0]['name']
    disease_node_id = disease_node[0]['_id']

    # Extract direct relationships
    direct_rels = find_direct_relationships_aql(db, drug_node_id, disease_node_id)
    
    # Extract intermediate paths
    paths = find_intermediate_nodes_aql(db, drug_node_id, disease_node_id)

    # Format the relationships
    relationships = []
    for rel in direct_rels:
        if '_id' in rel['edge']:
            rel_type = rel['edge']['_id'].split('/')[0]  # Get collection name as relation type
            relationships.append({
                'type': 'direct',
                'relation': rel_type,
                'details': rel['edge']
            })

    for path in paths:
        if 'edges' in path and path['edges']:
            for edge in path['edges']:
                if '_id' in edge:
                    rel_type = edge['_id'].split('/')[0]
                    relationships.append({
                        'type': 'intermediate',
                        'relation': rel_type,
                        'details': edge
                    })

    # Generate text description and explanation
    description = f"Analysis of relationship between {drug_name} and {disease_name}:\n"
    for rel in relationships:
        description += f"- {rel['type']} relationship: {rel['relation']}\n"

    prompt = f"Explain why {drug_name} could be effective for treating {disease_name} based on these relationships:\n{description}"
    explanation = generate_explanation(prompt)

    return {
        "drug_info": {
            "id": drug_node_id,
            "name": drug_name
        },
        "disease_info": {
            "id": disease_node_id,
            "name": disease_name
        },
        "relationships": relationships,
        "description": description,
        "explanation": explanation
    }

In [11]:
def find_direct_relationships(db, drug_node, disease_node):
    """
    Find direct relationships between drug and disease nodes.

    :param db: ArangoDB database connection object
    :param drug_node: ID of the drug node
    :param disease_node: ID of the disease node
    :return: List of direct relationships
    """
    direct_rels = nx.shortest_path(G, source=drug_node, target=disease_node)
    return direct_rels

def find_intermediate_nodes(db, drug_node, disease_node):
    
    """
    Find paths with intermediate nodes between drug and disease nodes.

    :param db: ArangoDB database connection object
    :param drug_node: ID of the drug node
    :param disease_node: ID of the disease node
    :return: List of paths with intermediate nodes
    """
    intermediate_paths = nx.all_shortest_paths(G, source=drug_node, target=disease_node)
    return intermediate_paths

def extract_subgraph(db, drug_node, disease_node):
    """
    Extract subgraph containing the drug, disease, and connecting paths with important metadata.

    :param db: ArangoDB database connection object
    :param drug_node: ID of the drug node
    :param disease_node: ID of the disease node
    :return: NetworkX graph object containing the subgraph
    """
    # Initialize an empty graph
    subgraph = nx.Graph()

    # Find direct relationships
    direct_relationships = find_direct_relationships(db, drug_node, disease_node)
    for relationship in direct_relationships:
        subgraph.add_edge(drug_node, disease_node, **relationship['edge'])

    # Find intermediate paths
    intermediate_paths = find_intermediate_nodes(db, drug_node, disease_node)
    for path in intermediate_paths:
        vertices = path['vertices']
        edges = path['edges']
        for i in range(len(vertices) - 1):
            subgraph.add_edge(vertices[i]['_id'], vertices[i + 1]['_id'], **edges[i])

    return subgraph

def find_intermediate_nodes_aql(db, drug_node, disease_node):
    
    """
    Find paths with intermediate nodes between drug and disease nodes.

    :param db: ArangoDB database connection object
    :param drug_node: ID of the drug node
    :param disease_node: ID of the disease node
    :return: List of paths with intermediate nodes
    """
    aql_query = """
    FOR v, e, p IN 1..2 ANY @drug_node GRAPH 'DRKG'
        FILTER p.vertices[-1]._id == @disease_node
        RETURN p
    """
    bind_vars = {
        "drug_node": drug_node,
        "disease_node": disease_node
    }
    cursor = db.aql.execute(aql_query, bind_vars=bind_vars)
    return [doc for doc in cursor]

def find_direct_relationships_aql(db, drug_node, disease_node):
    """
    Find direct relationships between drug and disease nodes.

    :param db: ArangoDB database connection object
    :param drug_node: ID of the drug node
    :param disease_node: ID of the disease node
    :return: List of direct relationships
    """
    aql_query = """
    FOR v, e IN 1..1 ANY @drug_node GRAPH 'DRKG'
        FILTER v._id == @disease_node
        RETURN {edge: e, vertex: v}
    """
    bind_vars = {
        "drug_node": drug_node,
        "disease_node": disease_node
    }
    cursor = db.aql.execute(aql_query, bind_vars=bind_vars)
    return [doc for doc in cursor]


In [12]:
@tool
def analyze_relationship(query: str) -> str:
    """
    Analyzes the relationship between entities mentioned in the query.

    Args:
        query (str): Natural language query about relationship between two entities

    Returns:
        str: Description of found relationships
    """
    # Extract entity names from query
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system", 
                "content": "Extract the two entity names from the relationship query. Return them as 'entity1: name1, entity2: name2'"
            },
            {
                "role": "user",
                "content": query
            }
        ],
        temperature=0.7
    )
    
    # Parse response to get entity names
    result = response.choices[0].message.content
    entity1 = result.split(',')[0].split(':')[1].strip()
    entity2 = result.split(',')[1].split(':')[1].strip()
    
    # Search for entities in database
    entity1_results = search_arangodb_view(
        db=db,
        view_name="node-search",
        search_fields=["name", "Identifier"],
        search_term=entity1,
        limit=1
    )
    
    entity2_results = search_arangodb_view(
        db=db,
        view_name="node-search", 
        search_fields=["name", "Identifier"],
        search_term=entity2,
        limit=1
    )
    
    if not entity1_results or not entity2_results:
        return f"Could not find one or both entities in database."
    
    intermediate_paths = find_intermediate_nodes(
        db,
        entity1_results[0]['Identifier'],
        entity2_results[0]['Identifier']
    )
    
    print(intermediate_paths)
    # Format response
    response_parts = [f"Analysis of relationship between {entity1} and {entity2}:"]
    
    # Add intermediate paths
    if intermediate_paths:
        for path in intermediate_paths:
            print(f"\nPath: {path}")
            
            # Get node attributes
            node_data = {node: G.nodes[node] for node in path}

            # Get edge attributes
            edge_data = {(path[i], path[i+1]): G.get_edge_data(path[i], path[i+1]) for i in range(len(path)-1)}

            response_parts.append(f"\nPath: {path}")
            response_parts.append(f"Node Attributes: {node_data}")
            response_parts.append(f"Edge Attributes: {edge_data}")
    
    if not intermediate_paths:
        response_parts.append("\nNo relationships found between these entities.")
        
    return "\n".join(response_parts)


In [13]:
def _create_graph_visualization(subgraph_nodes=None, subgraph_edges=None, max_nodes=100, 
                            highlight_nodes=None, title="Knowledge Graph Visualization",
                            output_format="html"):
    """
    Create a visualization of the graph or a subgraph using Plotly, display it in a notebook
    and save it to a file.

    Args:
        subgraph_nodes (list, optional): List of node IDs to include in visualization. 
                                        If None, uses full graph (limited by max_nodes).
        subgraph_edges (list, optional): List of (source, target) edges to include.
                                        If None, includes all edges between subgraph_nodes.
        max_nodes (int): Maximum number of nodes to include if using the full graph.
        highlight_nodes (list, optional): List of node IDs to highlight with a larger marker and different color.
        title (str): Title for the visualization.
        output_format (str): Format of the output file ("html" or "png").

    Returns:
        tuple: (Plotly figure object, path to the saved file)
    """
    # Create a color map for node types
    node_types = set()
    for node_id, attrs in G.nodes(data=True):
        if 'type' in attrs:
            node_types.add(attrs['type'])

    color_palette = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
        '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
    ]

    color_map = {node_type: color_palette[i % len(color_palette)] for i, node_type in enumerate(node_types)}
    
    # Special color for highlighted/queried nodes
    highlight_color = '#FF0000'  # Bright red for highlighted nodes

    nodes_to_viz = set(subgraph_nodes) if subgraph_nodes else set(list(G.nodes())[:max_nodes])
    subgraph = G.subgraph(nodes_to_viz)

    edges_to_viz = subgraph_edges if subgraph_edges else [(u, v, data) for u, v, data in subgraph.edges(data=True)]

    minigraph = nx.Graph()
    for node_id in nodes_to_viz:
        if node_id in G:  # Ensure node exists in original graph
            minigraph.add_node(node_id, **G.nodes[node_id])

    for source, target, data in edges_to_viz:
        if source in minigraph and target in minigraph:  # Ensure both endpoints exist
            minigraph.add_edge(source, target, **(data if isinstance(data, dict) else {}))

    pos = nx.spring_layout(minigraph, seed=42)

    node_x, node_y, node_text, node_color, node_size = [], [], [], [], []
    for node_id in minigraph.nodes():
        x, y = pos[node_id]
        node_x.append(x)
        node_y.append(y)
        
        node_attrs = minigraph.nodes[node_id]
        node_name = node_attrs.get('name', node_id)
        node_name = str(node_name)[:17] + "..." if len(str(node_name)) > 20 else str(node_name)
        
        node_type = node_attrs.get('type', 'Unknown')
        node_text.append(f"{node_name}<br>({node_type})")
        
        # Use highlight color for queried nodes, otherwise use the type color
        if highlight_nodes and node_id in highlight_nodes:
            node_color.append(highlight_color)
            node_size.append(25)  # Larger size for highlighted nodes
        else:
            node_color.append(color_map.get(node_type, '#CCCCCC'))
            node_size.append(15)  # Regular size for other nodes

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        text=node_text,
        marker=dict(showscale=False, color=node_color, size=node_size, line=dict(width=1, color='#888'))
    )

    edge_x, edge_y = [], []
    for source, target, _ in minigraph.edges(data=True):
        x0, y0 = pos[source]
        x1, y1 = pos[target]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=title,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                    ))

    # Add legend entries for node types
    for node_type, color in color_map.items():
        fig.add_trace(go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=10, color=color),
            name=node_type,
            showlegend=True
        ))
    
    # Add legend entry for highlighted/queried nodes
    if highlight_nodes:
        fig.add_trace(go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color=highlight_color),
            name='Queried Node',
            showlegend=True
        ))
    
    # Save to file
    temp_dir = tempfile.gettempdir()
    file_extension = "html" if output_format == "html" else "png"
    output_path = os.path.join(temp_dir, f"graph_viz_{random.randint(1000, 9999)}.{file_extension}")

    try:
        if output_format == "html":
            fig.write_html(output_path)
        elif output_format == "png":
            fig.write_image(output_path, format="png", scale=2)
        
        # Return both the figure (for display) and the path (for reference)
        return fig, output_path
    except Exception as e:
        return None, f"Error generating visualization: {str(e)}"

def _extract_visualization_request(query: str):
    """
    Extract visualization request type and entities using OpenAI.

    Args:
        query (str): The visualization query
        
    Returns:
        tuple: (request_type, entities)
            - request_type: 'full', 'subgraph', or 'path'
            - entities: list of entity names or None
    """
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "system",
            "content": """
            Extract visualization request information from the query.
            If the query is about a full graph visualization, return: full, None
            If the query is about a subgraph visualization, return: subgraph, [entity_name]
            If the query is about a path visualization, return: path, [entity1_name, entity2_name]
            """
        }, {
            "role": "user",
            "content": query
        }]
    )

    result = response.choices[0].message.content.strip()
    parts = result.split(',', 1)

    request_type = parts[0].strip().lower()
    entities = None

    if len(parts) > 1 and parts[1].strip().lower() != 'none':
        # Parse the entities part
        entities_str = parts[1].strip()
        
        # Handle list notation if present
        if entities_str.startswith('[') and entities_str.endswith(']'):
            entities_str = entities_str[1:-1]
            
        # Split by comma for multiple entities
        if ',' in entities_str:
            entities = [e.strip() for e in entities_str.split(',')]
        else:
            entities = [entities_str.strip()]

    return request_type, entities

@tool
def visualize_graph(query: str):
    """
    Tool function for visualizing the knowledge graph or subgraphs based on queries.
    Both displays the visualization in the notebook and saves it to a file.

    Args:
        query (str): Query specifying what to visualize. Can be:
                    - "full": Visualize a sample of the full graph
                    - "subgraph [entity name]": Visualize a local subgraph around an entity
                    - "path [entity1] [entity2]": Visualize paths between two entities
        output_format (str): Format to save the file in ("html" or "png")

    Returns:
        Plotly figure object that will display in the notebook
    """
    output_format="html"    
    # Extract request type and entities using OpenAI
    request_type, entities = _extract_visualization_request(query)

    # Handle full graph visualization
    if request_type == "full":
        fig, filepath = _create_graph_visualization(
            max_nodes=50, 
            title="Knowledge Graph Sample", 
            output_format=outp
        )
        
    # Handle subgraph visualization
    elif request_type == "subgraph" and entities and len(entities) == 1:
        entity_name = entities[0]
        
        # Find the entity in the graph
        entity_nodes = search_arangodb_view(
            db=db,
            view_name="node-search",
            search_fields=["name", "Identifier"],
            search_term=entity_name,
            limit=5
        )
        
        if not entity_nodes:
            return HTML(f"<div style='color:red'>Entity '{entity_name}' not found in the graph.</div>")
        
        entity_id = entity_nodes[0]['_id']
        
        # Get the ego network (node and its neighbors)
        if entity_id in G:
            neighbors = list(G.neighbors(entity_id))
            subgraph_nodes = [entity_id] + neighbors
            
            fig, filepath = _create_graph_visualization(
                subgraph_nodes=subgraph_nodes,
                highlight_nodes=[entity_id],
                title=f"Subgraph around {entity_name}",
                output_format=output_format
            )
        else:
            return HTML(f"<div style='color:red'>Entity ID '{entity_id}' not found in the NetworkX graph.</div>")

    # Handle path visualization
    elif request_type == "path" and entities and len(entities) == 2:
        entity1_name = entities[0]
        entity2_name = entities[1]
        
        # Find the entities in the graph
        entity1_nodes = search_arangodb_view(
            db=db,
            view_name="node-search",
            search_fields=["name", "Identifier"],
            search_term=entity1_name,
            limit=5
        )

        entity2_nodes = search_arangodb_view(
            db=db,
            view_name="node-search",
            search_fields=["name", "Identifier"],
            search_term=entity2_name,
            limit=5
        )
        
        if not entity1_nodes:
            return HTML(f"<div style='color:red'>Entity '{entity1_name}' not found.</div>")
        if not entity2_nodes:
            return HTML(f"<div style='color:red'>Entity '{entity2_name}' not found.</div>")
        
        entity1_id = entity1_nodes[0]['Identifier']
        entity2_id = entity2_nodes[0]['Identifier']
        
        # Check if both nodes exist in the graph
        if entity1_id not in G or entity2_id not in G:
            return HTML("<div style='color:red'>One or both entities not found in the NetworkX graph.</div>")
        
        # Find all paths between the nodes (limited to length 3 for performance)
        try:
            paths = list(nx.all_simple_paths(G, entity1_id, entity2_id, cutoff=3))
        except nx.NetworkXNoPath:
            return HTML(f"<div style='color:red'>No paths found between '{entity1_name}' and '{entity2_name}'.</div>")
        
        if not paths:
            return HTML(f"<div style='color:red'>No paths found between '{entity1_name}' and '{entity2_name}'.</div>")
        
        # Collect all nodes and edges in the paths
        path_nodes = set()
        path_edges = []
        
        for path in paths:
            for node in path:
                path_nodes.add(node)
            
            for i in range(len(path) - 1):
                path_edges.append((path[i], path[i+1], G.get_edge_data(path[i], path[i+1])))
        
        fig, filepath = _create_graph_visualization(
            subgraph_nodes=list(path_nodes),
            subgraph_edges=path_edges,
            highlight_nodes=[entity1_id, entity2_id],
            title=f"Paths between {entity1_name} and {entity2_name}",
            output_format=output_format
        )

    else:
        return HTML("""
        <div style='color:red'>
        Could not understand visualization request. Try:<br>
        - 'Show me the full graph'<br>
        - 'Show me a subgraph around [entity]'<br>
        - 'Show me paths between [entity1] and [entity2]'
        </div>
        """)
    
    # If we got a figure and a filepath, display the figure and show a note about the file
    if fig is not None and isinstance(filepath, str):
        # Add a note to the figure's title about the saved file
        fig.update_layout(
            title=f"{fig.layout.title.text} (Saved to: {filepath})"
        )
        
        # In a Jupyter notebook, returning the figure will display it
        return fig
    
    # If we only got an error message
    if isinstance(filepath, str) and filepath.startswith("Error"):
        return HTML(f"<div style='color:red'>{filepath}</div>")
    
    # Fallback return
    return fig

### Getting the database ready

The graph might take about 6-7 minutes to load please wait patiently 

In [ ]:
print("Please wait for about 6 minutes while we load the knowledge graph data...")
G = load_graph_data(NODES_PATH, EDGES_PATH)

db = create_arangodb_connection(HOSTS, USERNAME, PASSWORD)

Please wait for about 6 minutes while we load the knowledge graph data...
View 'node-search' already exists.
Linked collection 'DRKG_node' to view 'node-search' with indexed fields: {'name': ['text_en'], 'Identifier': ['text_en']}


In [15]:
def pagerank_around_node(target_node,graph = G, depth=1):
    """
    Compute PageRank scores for nodes in the neighborhood of a target node.

    Parameters:
    - graph (networkx.Graph or networkx.DiGraph): The input graph.
    - target_node: The node around which to compute PageRank.
    - depth (int): The depth of the neighborhood (1 for direct neighbors, 2 for neighbors of neighbors, etc.).

    Returns:
    - pagerank_scores (dict): A dictionary of PageRank scores for nodes in the subgraph.
    - subgraph (networkx.Graph or networkx.DiGraph): The subgraph used for the computation.
    """
    # Step 1: Extract the subgraph around the target node
    neighborhood = nx.single_source_shortest_path_length(graph, target_node, depth)
    subgraph = graph.subgraph(neighborhood.keys())

    # Step 2: Compute PageRank on the subgraph
    pagerank_scores = nx.pagerank(subgraph)

    return pagerank_scores, subgraph

### Create the agent

In [16]:
# Initialize the model
model = ChatOpenAI(temperature=1.0, api_key=op_pwd)
arangogr = ArangoGraph(db)

# Initialize the graph QA chain
graph_qa_chain = ArangoGraphQAChain.from_llm(
    llm=model,
    graph=arangogr,
    allow_dangerous_requests=True,
    verbose=True
)

# Store the original visualization function
original_visualize_graph = visualize_graph

# Global variable to store the last visualization
last_visualization = None

# Wrapper for the visualize_graph tool to capture its outputs
def visualize_graph_wrapper(query):
    """Wrapper that captures the visualization output"""
    result = original_visualize_graph(query)
    # Store the last result in a global variable
    global last_visualization
    last_visualization = result
    return result

# Define the tools with the wrapper
tools = [
    drug_repurposing,
    analyze_relationship,
    Tool(
        name="visualize_graph",
        func=visualize_graph_wrapper,
        description="Useful for visualizing the knowledge graph or subgraphs based on queries"
    ),
    Tool(
        name="Graph QA",
        func=graph_qa_chain.run,
        description="Useful for querying the knowledge graph about relationships between entities"
    )
]

# Custom output parser to handle visualization objects
class DirectVisualizationOutputParser(AgentOutputParser):
    def parse(self, llm_output):
        # Check if we have a visualization request
        if "Action: visualize_graph" in llm_output:
            action_input_match = re.search(r'Action Input: "(.*?)"', llm_output)
            if action_input_match:
                visualization_query = action_input_match.group(1)
                return AgentAction(tool="visualize_graph", tool_input=visualization_query, log=llm_output)
       
        # Check if we have a final answer
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
           
        # Standard parsing for other actions
        action_match = re.search(r'Action: (.*?)[\n]', llm_output)
        action_input_match = re.search(r'Action Input: (.*)', llm_output)
       
        if action_match and action_input_match:
            action = action_match.group(1).strip()
            action_input = action_input_match.group(1).strip()
            # Remove quotes if present
            if action_input.startswith('"') and action_input.endswith('"'):
                action_input = action_input[1:-1]
            return AgentAction(tool=action, tool_input=action_input, log=llm_output)
       
        # If no action or final answer is found
        raise ValueError(f"Could not parse LLM output: {llm_output}")

# Initialize the agent with the custom parser
agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callback_manager=CallbackManager([BaseCallbackHandler()]),
    verbose=True,
    agent_kwargs={"output_parser": DirectVisualizationOutputParser()}
)

# Function to create biological explanation prompt
def create_bio_prompt(agent_output):
    return f"""
    Based on the following information, explain the relationships
    between the entities in biological terms, focusing on mechanisms,
    pathways, and physiological relevance:
   
    {agent_output}
    """

# Create the final chain for biological interpretations
final_chain = (
    agent
    | RunnablePassthrough.assign(bio_prompt=create_bio_prompt)
    | (lambda x: model.predict(x["bio_prompt"]))
)

# Integrated function that handles both visualization and regular queries
def integrated_query_handler(query):
    """
    Handles queries and returns either visualizations or processed results
    from the final_chain depending on which tool was used.
    """
    # Reset the last visualization
    global last_visualization
    last_visualization = None
    
    # Run the agent
    try:
        # For direct visualization requests, use the agent directly
        if "visualize" in query.lower() or "visualization" in query.lower() or "graph" in query.lower():
            agent_output = agent.run(query)
            
            # If a visualization was generated, return it
            if last_visualization is not None:
                return last_visualization
            return agent_output
        else:
            # For other requests, use the final chain with biological interpretation
            return final_chain.invoke({"input": query})
    except Exception as e:
        return f"Error processing query: {str(e)}"

# Usage example
# result = integrated_query_handler("Show me the relationship between ACE2 and COVID-19")

/tmp/ipykernel_875/1430047230.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature=1.0, api_key=op_pwd)
/tmp/ipykernel_875/1430047230.py:77: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_

### Examples

In [17]:
result = integrated_query_handler("Analyze the relationship between ACE2 and coronavirus and explain the mechanism")



> Entering new AgentExecutor chain...
I should use the tool analyze_relationship to understand the relationship between ACE2 and coronavirus and their mechanism.
Action: analyze_relationship
Action Input: "What is the relationship between ACE2 and coronavirus and what is their mechanism?"<generator object _build_paths_from_predecessors at 0x7f4941bf9f20>

Path: ['Gene::59272', 'Compound::DB00499', 'Compound::DB00811', 'Disease::MESH:D018352']

Path: ['Gene::59272', 'Compound::DB00722', 'Compound::DB00811', 'Disease::MESH:D018352']

Path: ['Gene::59272', 'Disease::MESH:D007674', 'Compound::DB00811', 'Disease::MESH:D018352']

Path: ['Gene::59272', 'Disease::MESH:D011014', 'Compound::DB00811', 'Disease::MESH:D018352']

Path: ['Gene::59272', 'Disease::MESH:D002318', 'Compound::MESH:C059514', 'Disease::MESH:D018352']

Path: ['Gene::59272', 'Disease::MESH:D002544', 'Compound::MESH:C059514', 'Disease::MESH:D018352']

Path: ['Gene::59272', 'Disease::MESH:D055371', 'Compound::MESH:C059514', '

In [18]:
print(result)

ACE2, also known as angiotensin-converting enzyme 2, is a protein found on the surface of cells in the human body. It serves as the entry point for coronaviruses, including the virus responsible for COVID-19, to infect host cells. This interaction between ACE2 and the coronavirus is essential for the virus to invade and replicate within the host.

The mechanism of this relationship involves the spike protein on the surface of the coronavirus binding to ACE2 on the host cell. This binding allows the virus to enter the cell and hijack its machinery to produce more viruses. The genetic interactions between the viral RNA and host cell DNA further drive the replication and spread of the virus within the body.

Moreover, the compound interactions between ACE2 and the virus also play a crucial role in the pathogenesis of coronavirus infections. By targeting ACE2, researchers are exploring potential therapeutic strategies to block the virus from entering host cells or interfering with viral re

In [19]:
result = integrated_query_handler("visualize the relationship between diabetes and Glyburide")

# Display the result
display(result)



> Entering new AgentExecutor chain...
/tmp/ipykernel_875/1430047230.py:117: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_output = agent.run(query)
I should use the visualize_graph tool to create a visualization of the relationship between diabetes and Glyburide.
Action: visualize_graph
Action Input: "diabetes AND Glyburide"/tmp/ipykernel_875/1430047230.py:22: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = original_visualize_graph(query)

Observation: Figure({
    'data': [{'hoverinfo': 'none',
              'line': {'color': '#888', 'width': 1},
              'mode': 'lines',
              'type': 'scatter',
              'x': [0.08832067251205444, -0.06397710740566254, None, ...,
                    0.01894611120223999, 0.19701938331127167, None],
            

In [20]:
result = integrated_query_handler("What is the relationship between diabetes and Glyburide")



> Entering new AgentExecutor chain...
I can use the tools available to analyze the relationship between diabetes and Glyburide.
Action: analyze_relationship
Action Input: "Diabetes and Glyburide"<generator object _build_paths_from_predecessors at 0x7f4941bf9df0>

Path: ['Disease::MESH:D048909', 'Gene::3630', 'Compound::DB01016']

Path: ['Disease::MESH:D048909', 'Compound::DB11672', 'Compound::DB01016']

Observation: Analysis of relationship between Diabetes and Glyburide:

Path: ['Disease::MESH:D048909', 'Gene::3630', 'Compound::DB01016']
Node Attributes: {'Disease::MESH:D048909': {}, 'Gene::3630': {}, 'Compound::DB01016': {}}
Edge Attributes: {('Disease::MESH:D048909', 'Gene::3630'): {'Relation': 'GNBR::J::Gene:Disease', 'Connected entity-types': 'Disease:Gene', 'Interaction-type': 'role in pathogenesis', 'Relation_id': 45, '_from': 'DRKG_node/1789', '_to': 'DRKG_node/46740', '_key': '732886'}, ('Gene::3630', 'Compound::DB01016'): {'Relation': 'GNBR::K::Compound:Gene', 'Connected en

In [21]:
print(result)

In biological terms, diabetes is a metabolic disorder characterized by high levels of glucose in the blood due to either insufficient insulin production or the body's inability to respond effectively to insulin. Glyburide is a medication commonly used to treat type 2 diabetes by stimulating insulin release from pancreatic beta cells.

The relationship between diabetes and Glyburide revolves around treatment and therapy. Glyburide helps to lower blood glucose levels in individuals with diabetes by increasing insulin secretion. This mechanism of action helps to improve blood sugar control and reduce the risk of complications associated with diabetes, such as nerve damage, kidney disease, and cardiovascular problems.

Additionally, the relationship between diabetes and Glyburide also involves drug-drug interactions. It is important to consider potential interactions between Glyburide and other medications that a diabetic individual may be taking to manage their condition. Certain drugs ca

In [22]:
result = integrated_query_handler("What are possible drug repourposing candidates for coronavirus")



> Entering new AgentExecutor chain...
I should use the drug_repurposing tool to identify potential drug repurposing candidates for coronavirus.
Action: drug_repurposing
Action Input: "coronavirus"Coronavirus
[{'_key': '46703', '_id': 'DRKG_node/46703', '_rev': '_jTcOJ6---Q', 'name': 'Coronavirus Infections', 'id': 46703, 'Identifier': 'Disease::MESH:D018352', 'type': 'Disease'}, {'_key': '9394', '_id': 'DRKG_node/9394', '_rev': '_jTcOIcC-AT', 'name': 'Severe acute respiratory syndrome-related coronavirus', 'id': 9394, 'Identifier': 'Disease::MESH:D045473', 'type': 'Disease'}, {'_key': '9425', '_id': 'DRKG_node/9425', '_rev': '_jTcOIcC-CI', 'name': 'Middle East Respiratory Syndrome Coronavirus', 'id': 9425, 'Identifier': 'Disease::MESH:D065207', 'type': 'Disease'}, {'_key': '9430', '_id': 'DRKG_node/9430', '_rev': '_jTcOIcC-Cd', 'name': 'Coronavirus 229E, Human', 'id': 9430, 'Identifier': 'Disease::MESH:D028941', 'type': 'Disease'}, {'_key': '9431', '_id': 'DRKG_node/9431', '_rev': '_

In [23]:
print(result)

Error processing query: name 'analyze_drug_disease_relationship_aql' is not defined


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=129f09a4-04c4-4e2b-a317-686d4c775f2c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>